### Name:
> Castillo, Marvien Angel C. <br>
> Herrera, Mikhaela Gabrielle B. <br>
> Regindin, Sean Adrien I. <br>

# Setup Environment

In [4]:
import os

# Add the directory containing the executable to the PATH
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

# Check if the directory is added to the PATH
print(os.environ["PATH"])

/opt/tljh/user/bin:/bin:/usr/bin:/usr/local/cuda/bin


# Check if CUDA is present

In [5]:
%%bash
nvcc --version
nvprof --version
nsys --version
ncu --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Wed_Apr__9_19:24:57_PDT_2025
Cuda compilation tools, release 12.9, V12.9.41
Build cuda_12.9.r12.9/compiler.35813241_0
nvprof: NVIDIA (R) Cuda command line profiler
Copyright (c) 2012 - 2025 NVIDIA Corporation
Release version 12.9.19 (21)
NVIDIA Nsight Systems version 2025.1.3.140-251335620677v0
NVIDIA (R) Nsight Compute Command Line Profiler
Copyright (c) 2018-2025 NVIDIA Corporation
Version 2025.2.0.0 (build 35613519) (public-release)


In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Nov  4 06:55:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 575.51.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           Off |   00000000:00:10.0 Off |                    0 |
| N/A   27C    P0             22W /  250W |       0MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Variant 1 - C Program

In [61]:
%%writefile C_var1.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
// ***C function version
void kernel_C(float A[], float B[], float C[], size_t n, int idx[]) {
	for (int i = 0; i < n; i++) {
		if (A[i] >= B[i]) {
			C[i] = A[i];
			idx[i] = 0;
		}
		else {
			C[i] = B[i];
			idx[i] = 1;
		}
	}
}

int main(int argc, char** argv)
{
   const size_t ARRAY_SIZE = 1<<24;
   const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
   const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
//number of times the program is to be executed
   const size_t loope = 10;
//declare array
   float *C,*A,*B;
   int *idx,a;
   A = (float*)malloc(FLOAT_ARRAY_BYTES);
   B = (float*)malloc(FLOAT_ARRAY_BYTES);
   C = (float*)malloc(FLOAT_ARRAY_BYTES);
   idx = (int*)malloc(INT_ARRAY_BYTES);
   a=2;
//timer variables
  clock_t start, end;
// ***--- initialize your array here ---------
   int i;
	for (i = 0; i < ARRAY_SIZE; i++) {
		A[i] = sin(i * 0.0005) * 100.0 + 50.0;
		B[i] = cos(i * 0.0003) * 100.0 + 50.0;
	}
// fill-in cache
    kernel_C(A,B,C,ARRAY_SIZE,idx);
//time here
  double elapse, time_taken;
  elapse = 0.0f;
  for (int i=0; i<loope; i++){
    start = clock();
      kernel_C(A,B,C,ARRAY_SIZE,idx );
    end = clock();
    time_taken = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
    elapse = elapse + time_taken;
  }
  printf("Function (in C) average time for %lu loops is %f milliseconds to execute an array size %lu \n", loope, elapse/loope, ARRAY_SIZE);

// error checking routine here --
   size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
   printf("Error count (C program): %lu\n", err_count);
  // Free memory
  free(A);
  free(B);
  free(C);
  free(idx);
  return 0;
}

Overwriting C_var1.c


In [62]:
%%bash
gcc C_var1.c -o C_var1 -lm

In [63]:
%%bash
./C_var1

Function (in C) average time for 10 loops is 154.756900 milliseconds to execute an array size 16777216 
Error count (C program): 0


# Variant 2 - Grid Stride Loop

In [64]:
%%writefile CUDA_var2.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
//Grid stride loop

//*** CUDA square kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);
// *** init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }

// *** setup CUDA kernel
  size_t numThreads = 1024;
  size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function = Float kernel_C\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks,numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();
//error checking
    size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
  printf("Error count(CUDA program): %zu\n", err_count);
//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}


Overwriting CUDA_var2.cu


In [65]:
%%bash
nvcc CUDA_var2.cu -o CUDA_var2 -Wno-deprecated-gpu-targets

In [66]:
%%bash
nvprof ./CUDA_var2

==989565== NVPROF is profiling process 989565, command: ./CUDA_var2


*** function = Float kernel_C
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Error count(CUDA program): 0


==989565== Profiling application: ./CUDA_var2
==989565== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  100.97ms        10  10.097ms  343.74us  97.869ms  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   91.37%  1.26708s         4  316.77ms  41.914us  1.26641s  cudaMallocManaged
                    7.28%  100.92ms         1  100.92ms  100.92ms  100.92ms  cudaDeviceSynchronize
                    1.17%  16.266ms         4  4.0666ms  3.1726ms  4.7747ms  cudaFree
                    0.12%  1.6242ms        10  162.42us  7.8420us  1.4983ms  cudaLaunchKernel
                    0.04%  558.32us       114  4.8970us     114ns  214.37us  cuDeviceGetAttribute
                    0.01%  200.77us         1  200.77us  200.77us  200.77us  cuDeviceGetName
                    0.00%  37.949us         1  37.949us  37.949us  37.949us  cuDeviceTotalMem
                    0.00%  8.9970us         1  8.9970

# Variant 3.0 - Grid Stride Loop with Prefetch

In [69]:
%%writefile CUDA_var3.cu
// prefetch
#include <stdio.h>
#include <stdlib.h>

//CUDA square kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }
 //"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize(); 

//"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);

//error checking
    size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
  printf("Error count(CUDA program): %zu\n", err_count);

//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}

Writing CUDA_var3.cu


In [70]:
%%bash
nvcc CUDA_var3.cu -o CUDA_var3 -Wno-deprecated-gpu-targets

In [71]:
%%bash
nvprof ./CUDA_var3

==989787== NVPROF is profiling process 989787, command: ./CUDA_var3


*** function ***
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Error count(CUDA program): 0


==989787== Profiling application: ./CUDA_var3
==989787== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.5442ms        10  354.42us  352.61us  358.17us  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   83.27%  554.27ms         4  138.57ms  45.667us  553.88ms  cudaMallocManaged
                   11.78%  78.433ms         8  9.8042ms  6.6360us  31.281ms  cudaMemPrefetchAsync
                    2.50%  16.653ms         4  4.1633ms  2.4399ms  8.7312ms  cudaFree
                    1.86%  12.362ms        10  1.2362ms  4.9500us  12.301ms  cudaLaunchKernel
                    0.52%  3.4887ms         1  3.4887ms  3.4887ms  3.4887ms  cudaDeviceSynchronize
                    0.05%  308.42us       114  2.7050us     213ns  118.57us  cuDeviceGetAttribute
                    0.01%  52.929us         1  52.929us  52.929us  52.929us  cuDeviceGetName
                    0.01%  44.551us         1  44

# Variant 4.0 - Grid Stride Loop with Prefetch and Page Creation

In [72]:
%%writefile CUDA_var4.cu
//prefetch + page creation
// page creation responsible gpu fault page
#include <stdio.h>
#include <stdlib.h>

//CUDA square kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}


int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);
//get gpu id
  int device = -1;
  cudaGetDevice(&device);

//"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
//"prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }
//"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;   
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();

  //"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);

//error checking
    size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
  printf("Error count(CUDA program): %zu\n", err_count);


//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}

Writing CUDA_var4.cu


In [73]:
%%bash
nvcc CUDA_var4.cu -o CUDA_var4 -Wno-deprecated-gpu-targets

In [74]:
%%bash
nvprof ./CUDA_var4

==989905== NVPROF is profiling process 989905, command: ./CUDA_var4


*** function ***
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Error count(CUDA program): 0


==989905== Profiling application: ./CUDA_var4
==989905== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.6561ms        10  365.61us  362.43us  372.86us  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   77.03%  542.82ms         4  135.70ms  31.108us  542.53ms  cudaMallocManaged
                   19.46%  137.11ms        12  11.426ms  15.127us  35.583ms  cudaMemPrefetchAsync
                    1.81%  12.746ms        10  1.2746ms  4.6960us  12.685ms  cudaLaunchKernel
                    1.12%  7.9101ms         4  1.9775ms  1.8914ms  2.1216ms  cudaFree
                    0.51%  3.5987ms         1  3.5987ms  3.5987ms  3.5987ms  cudaDeviceSynchronize
                    0.05%  339.91us       114  2.9810us     194ns  139.28us  cuDeviceGetAttribute
                    0.01%  56.592us         1  56.592us  56.592us  56.592us  cuDeviceGetName
                    0.01%  36.867us         1  36

# Variant 5.0 - Grid Stride Loop with Prefetch and Page Creation + mem advise

In [75]:
%%writefile CUDA_var5.cu
//prefetch + page creation + memadvise

#include <stdio.h>
#include <stdlib.h>

//CUDA square kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// memory advise
   cudaMemAdvise(A, FLOAT_ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(A, FLOAT_ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
   cudaMemAdvise(B, FLOAT_ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(B, FLOAT_ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

//"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
//"prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }


 //"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;   // what if numThreads>1024, numThreads<1024?
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();

  //"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);

//error checking
    size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
  printf("Error count(CUDA program): %zu\n", err_count);


//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;

}


Writing CUDA_var5.cu
